In [73]:
import google.generativeai 
import json
from openai import OpenAI
from pydantic import BaseModel, Field, ValidationError 
from typing import List, Optional, Union
import datetime
import re
import io
import pandas as pd 
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

google.generativeai.configure(api_key=os.getenv("GEMINI_API_KEY")) 
model = google.generativeai.GenerativeModel('gemini-2.0-flash')
deepseek_client = OpenAI(api_key=os.getenv("DEEPSEEK_API_KEY"), base_url="https://api.deepseek.com")
openai_client = OpenAI(api_key=os.getenv("OPENAI_AI_KEY"))



In [46]:
# --- Справочники ---

# Список возможных культур и их написаний
cultures_list = """
Наименования с/х культур:
Вика+Тритикале
Горох на зерно
Горох товарный
Гуар
Конопля
Кориандр
Кукуруза кормовая
Кукуруза семенная
Кукуруза товарная
Люцерна
Многолетние злаковые травы
Многолетние травы прошлых лет
Многолетние травы текущего года
Овес
Подсолнечник кондитерский
Подсолнечник семенной
Подсолнечник товарный
Просо
Пшеница озимая на зеленый корм
Пшеница озимая семенная
Пшеница озимая товарная (возможные сокр.: оз пш товарн, озимая пш тов)
Рапс озимый
Рапс яровой
Свекла сахарная
Сорго
Сорго кормовой
Сорго-суданковый гибрид
Соя семенная
Соя товарная
Чистый пар
Чумиза
Ячмень озимый
Ячмень озимый семенной
"""

# Список возможных полевых операций
operations_list = """
Наименования полевых работ:
1-я междурядная культивация
2-я междурядная культивация
Боронование довсходовое
Внесение минеральных удобрений
Выравнивание зяби
2-е Выравнивание зяби
Гербицидная обработка (только у свеклы может быть 1, 2, 3, 4, у остальных только 1)
1 Гербицидная обработка
2 Гербицидная обработка
3 Гербицидная обработка
4 Гербицидная обработка
Дискование
Дискование 2-е
Дискование 3-е
Инсектицидная обработка
Культивация (возможные сокр.: культ)
Пахота
Подкормка
2-я подкормка
Предпосевная культивация (возможные сокр.: Предп культ)
Прикатывание посевов
Сев
Сплошная культивация
Уборка
Функицидная обработка
Чизлевание
"""

departments_data_list = [
  {
    "Подразделение": "АОР",
    "ПУ": "Кавказ",
    "Отделения": [18, 19]
  },
  {
    "Подразделение": "АОР",
    "ПУ": "Север",
    "Отделения": [3, 7, 10, 20]
  },
  {
    "Подразделение": "АОР",
    "ПУ": "Центр",
    "Отделения": [1, 4, 5, 6, 9]
  },
  {
    "Подразделение": "АОР",
    "ПУ": "Юг",
    "Отделения": [11, 12, 16, 17]
  },
  {
    "Подразделение": "АОР",
    "ПУ": "Рассвет",
    "Отделения": []
  },
  {
    "Подразделение": "ТСК",
    "ПУ": None, # Используем None для "Нет ПУ"
    "Отделения": [] # Пустой список для "Нет отделения"
  },
  {
    "Подразделение": "АО Кропоткинское",
    "ПУ": None,
    "Отделения": []
  },
  {
    "Подразделение": "Восход",
    "ПУ": None,
    "Отделения": []
  },
  {
    "Подразделение": "Колхоз Прогресс",
    "ПУ": None,
    "Отделения": []
  },
  {
    "Подразделение": "Мир",
    "ПУ": None,
    "Отделения": []
  },
  {
    "Подразделение": "СП Коломейцево",
    "ПУ": None,
    "Отделения": []
  }
]

# Преобразуем Python список словарей в JSON строку с отступами для читаемости
departments_list = json.dumps(departments_data_list, indent=2, ensure_ascii=False)

DETAILED_AGRO_REPORT_PROMPT_TEMPLATE="""
Проанализируй следующее сообщение с отчетом о сельскохозяйственных работах:
---
{input_message}
---

Используй следующие справочники для распознавания терминов и определения значений:

СПИСОК КУЛЬТУР:
{cultures_list}

СПИСОК ОПЕРАЦИЙ:
{operations_list}

СПИСОК ПОДРАЗДЕЛЕНИЙ:
{departments_list}

ЗАДАЧА:
1.  Найди **все** уникальные комбинации (Операция, Культура, Подразделение), логически связанные в тексте сообщения.
2.  Для **каждой** такой комбинации извлеки связанные с ней числовые данные: "За день, га", "С начала операции, га", "Вал за день, ц", "Вал с начала, ц".
3.  Определи **дату** отчета. Сначала попробуй найти дату в тексте сообщения (например, "25.07", "25 июля"). Если дата в тексте не найдена, используй текущую дату: {current_date}.
4.  Верни результат СТРОГО в формате JSON-списка (`list`), где каждый элемент списка - это JSON-объект (`dict`), представляющий одну найденную комбинацию и её данные.

ИНСТРУКЦИИ ПО ИЗВЛЕЧЕНИЮ ДЛЯ КАЖДОГО ОБЪЕКТА В СПИСКЕ:
- "Дата": Дата отчета в формате YYYY-MM-DD (извлеченная из текста или текущая {current_date}).
- "Подразделение": Определи название подразделения для данной комбинации. Используй номер отделения (Отд), чтобы найти соответствующее подразделение в "СПИСКЕ ПОДРАЗДЕЛЕНИЙ". Если есть несколько "Отд", относящихся к одной записи, ориентируйся на первое или на общие данные по ПУ (Производственному участку), если они есть. Если номер отделения не указан или не найден, попробуй найти название подразделения явно в тексте рядом с операцией/культурой. Если определить не удается, используй null. Не включай ПУ или Отделение в название.
- "Операция": Определи полное название полевой работы из "СПИСКА ОПЕРАЦИЙ", основываясь на тексте, связанном с этой комбинацией (например, "Предп культ" -> "Предпосевная культивация"). Если не указана или не распознана для этой комбинации, используй null.
- "Культура": Определи полное название культуры из "СПИСКА КУЛЬТУР", основываясь на тексте, связанном с этой комбинацией (например, "оз пш" -> "Пшеница озимая"). Если не указана или не распознана для этой комбинации, используй null.
- "За день, га": Количество гектар за день (число перед '/'). Если есть данные по "Отд", суммируй их. Если есть данные "По Пу" и "Отд", используй данные "По Пу". Если не указано, используй null.
- "С начала операции, га": Общее количество гектар с начала операции (число после '/'). Если есть данные по "Отд", суммируй их. Если есть данные "По Пу" и "Отд", используй данные "По Пу". Если не указано, используй null.
- "Вал за день, ц": Валовый сбор за день в центнерах. Если не указано, используй null.
- "Вал с начала, ц": Валовый сбор с начала операции в центнерах. Если не указано, используй null.

ОБРАТИ ВНИМАНИЕ: 
1. Если в сообщении указано:
Пахота зяби под мн тр
По Пу 26/488
Отд 12 26/221
то бери информацию только по ПУ - так как это данные сразу по производственному участку, а все строчки где указано "Отд" относятся к отдельным отделениям. (которые суммируются в случае, если значение "По Пу" не указано)
2. Если в сообщении указано:
Диск к. Сил отд 7. 32/352
Пу- 484
Это значит, что отделение сделало 32 гектара за день, 352 с начала операции, а по производственному участку (ПУ) - 484 гектара. Значит надо выносить цифры 32 гектра за день и 484 гектара с начала операции.


ДОПОЛНИТЕЛЬНЫЕ ТРЕБОВАНИЯ:
- Результатом должен быть JSON-список (`list`). Каждый элемент списка - JSON-объект (`dict`).
- Если сообщение не содержит данных о работах или их не удается извлечь, верни пустой список `[]`.
- Возвращай ТОЛЬКО JSON список, без какого-либо дополнительного текста, комментариев или объяснений до или после JSON.

ТРЕБУЕМЫЙ ФОРМАТ JSON-Списка:
[
  {{
    "Дата": "YYYY-MM-DD", 
    "Подразделение": str | null,
    "Операция": str | null,
    "Культура": str | null,
    "За день, га": float | int | null,
    "С начала операции, га": float | int | null,
    "Вал за день, ц": float | int | null,
    "Вал с начала, ц": float | int | null
  }},
  {{
    "Дата": "YYYY-MM-DD", 
    "Подразделение": str | null,
    "Операция": str | null,
    "Культура": str | null,
    "За день, га": float | int | null,
    "С начала операции, га": float | int | null,
    "Вал за день, ц": float | int | null,
    "Вал с начала, ц": float | int | null
  }}
  // ... и так далее для каждой найденной комбинации
]

ПРИМЕР ОЖИДАЕМОГО ВЫВОДА для сообщения:
"Север 26.07\\nОтд7 пах с св 41/501\\nОтд20 пах с св 20/281 по пу 61/793\\nОтд 3 пах подс.60/231"
[
  {{
    "Дата": "2024-07-26", 
    "Подразделение": "АОР",
    "Операция": "Пахота", 
    "Культура": "Свекла сахарная", 
    "За день, га": 61, 
    "С начала операции, га": 793, 
    "Вал за день, ц": null,
    "Вал с начала, ц": null
  }},
  {{
    "Дата": "2024-07-26", 
    "Подразделение": "АОР",
    "Операция": "Пахота", 
    "Культура": "Подсолнечник товарный", 
    "За день, га": 60, 
    "С начала операции, га": 231, 
    "Вал за день, ц": null,
    "Вал с начала, ц": null
  }}
]
"""

In [8]:
def gemini_extract_reports(input_message: str,
                                    cultures_list: str,
                                    operations_list: str,
                                    departments_list: str,
                                    model,
                                    prompt_template: str) -> list[dict] | None: # Добавлен параметр prompt_template
    """
    Обрабатывает текстовое сообщение с агро-отчетом, находит все уникальные
    комбинации (Операция, Культура, Подразделение), извлекает для каждой
    детальные данные и дату, и возвращает результат в виде списка словарей,
    используя предоставленный шаблон промпта.

    Args:
        input_message: Строка с текстом сообщения.
        cultures_list: Строка со списком культур.
        operations_list: Строка со списком операций.
        departments_list: Строка со списком подразделений (JSON).
        model: Инициализированная модель Gemini.
        prompt_template: Строка с f-string шаблоном промпта.

    Returns:
        Список словарей (JSON-объектов), где каждый словарь представляет
        одну уникальную комбинацию и её данные, или None в случае ошибки.
    """

    current_date = datetime.date.today().isoformat()

    # --- Формирование Промпта из шаблона ---
    prompt = prompt_template.format(
        input_message=input_message,
        cultures_list=cultures_list,
        operations_list=operations_list,
        departments_list=departments_list,
        current_date=current_date
    )

    try:
        response = model.generate_content(
            contents=prompt, # Используем сформированный промпт
            generation_config=google.generativeai.types.GenerationConfig(
                candidate_count=1,
                temperature=0.2
            )
        )

        # Ожидаем JSON-список
        parsed_data = None
        json_string = response.text.strip()

        # Убираем ```json и ```, если они есть
        if json_string.startswith("```json"):
            json_string = json_string[7:-3].strip()
        elif json_string.startswith("```"): # На случай если просто ``` без json
                json_string = json_string[3:-3].strip()

        parsed_data = json.loads(json_string)

        # Убедимся, что результат - это список
        if not isinstance(parsed_data, list):
                print(f"Ошибка: Модель вернула не список, а {type(parsed_data)}.")
                print("Ответ модели был:")
                print(response.text)
                return None

        # Проверим и добавим дату, если модель её не включила в какой-либо из объектов
        for item in parsed_data:
            if not isinstance(item, dict):
                    print(f"Ошибка: Элемент в списке не является словарем: {item}")
                    continue
            if "Дата" not in item or item["Дата"] is None:
                # Пробуем извлечь дату из сообщения, если модель не смогла
                # (Этот блок можно усложнить для парсинга даты из input_message,
                # но пока оставим как есть - используем current_date если модель не дала)
                item["Дата"] = current_date # Используем текущую дату, если не найдена

        return parsed_data # Возвращаем список словарей

    except json.JSONDecodeError as e:
        print(f"\nОшибка парсинга JSON: {e}")
        print("Ответ модели был:")
        print(response.text)
        return None
    except Exception as e:
        print(f"\nПроизошла ошибка при вызове Gemini API или обработке ответа: {e}")
        if hasattr(response, 'prompt_feedback'):
                print(f"Prompt Feedback: {response.prompt_feedback}")
        if 'response' in locals() and hasattr(response, 'text'):
                print("Ответ модели был:")
                print(response.text)
        return None

In [48]:
def deepseek_extract_reports(input_message: str,
                                    cultures_list: str,
                                    client: str,
                                    operations_list: str,
                                    departments_list: str,
                                    prompt_template: str) -> list[dict] | None: # Добавлен параметр prompt_template
    """
    Обрабатывает текстовое сообщение с агро-отчетом, находит все уникальные
    комбинации (Операция, Культура, Подразделение), извлекает для каждой
    детальные данные и дату, и возвращает результат в виде списка словарей,
    используя предоставленный шаблон промпта.

    Args:
        input_message: Строка с текстом сообщения.
        cultures_list: Строка со списком культур.
        operations_list: Строка со списком операций.
        departments_list: Строка со списком подразделений (JSON).
        model: Инициализированная модель Gemini.
        prompt_template: Строка с f-string шаблоном промпта.

    Returns:
        Список словарей (JSON-объектов), где каждый словарь представляет
        одну уникальную комбинацию и её данные, или None в случае ошибки.
    """

    current_date = datetime.date.today().isoformat()

    # --- Формирование Промпта из шаблона ---
    prompt = prompt_template.format(
        input_message=input_message,
        cultures_list=cultures_list,
        operations_list=operations_list,
        departments_list=departments_list,
        current_date=current_date
    )

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
            {"role": "system", "content": "You are an AI assistant designed to extract structured data from agricultural reports according to specific instructions and format."},
            {"role": "user", "content": prompt}
            ],
            stream=False
        )
        # Ожидаем JSON-список
        parsed_data = None
        json_string = response.choices[0].message.content.strip()

        # Убираем ```json и ```, если они есть
        if json_string.startswith("```json"):
            json_string = json_string[7:-3].strip()
        elif json_string.startswith("```"): # На случай если просто ``` без json
                json_string = json_string[3:-3].strip()

        parsed_data = json.loads(json_string)

        # Убедимся, что результат - это список
        if not isinstance(parsed_data, list):
                print(f"Ошибка: Модель вернула не список, а {type(parsed_data)}.")
                print("Ответ модели был:")
                print(response.text)
                return None

        # Проверим и добавим дату, если модель её не включила в какой-либо из объектов
        for item in parsed_data:
            if not isinstance(item, dict):
                    print(f"Ошибка: Элемент в списке не является словарем: {item}")
                    continue
            if "Дата" not in item or item["Дата"] is None:
                # Пробуем извлечь дату из сообщения, если модель не смогла
                # (Этот блок можно усложнить для парсинга даты из input_message,
                # но пока оставим как есть - используем current_date если модель не дала)
                item["Дата"] = current_date # Используем текущую дату, если не найдена

        return parsed_data # Возвращаем список словарей

    except json.JSONDecodeError as e:
        print(f"\nОшибка парсинга JSON: {e}")
        print("Ответ модели был:")
        print(response.text)
        return None
    except Exception as e:
        print(f"\nПроизошла ошибка при вызове Gemini API или обработке ответа: {e}")
        if hasattr(response, 'prompt_feedback'):
                print(f"Prompt Feedback: {response.prompt_feedback}")
        if 'response' in locals() and hasattr(response, 'text'):
                print("Ответ модели был:")
                print(response.text)
        return None

In [10]:
def save_reports_to_excel(report_data: list[dict] | None, filename: str):
    """
    Принимает список словарей (результат работы gemini_extract_reports)
    и сохраняет его в файл Excel.

    Args:
        report_data: Список словарей с данными отчетов, или None, если данных нет.
        filename: Имя файла для сохранения (например, 'agro_report.xlsx').
    """
    # Проверяем, есть ли данные для сохранения
    if not report_data:
        print("Нет данных для сохранения в Excel. Список пуст или равен None.")
        return

    # Проверяем, что report_data действительно список словарей (хотя бы первый элемент)
    if not isinstance(report_data, list) or not all(isinstance(item, dict) for item in report_data):
         print(f"Ошибка: Ожидался список словарей, но получен тип {type(report_data)}. Сохранение отменено.")
         # Можно добавить вывод самого report_data для диагностики, если нужно
         # print("Полученные данные:", report_data)
         return

    try:
        # 1. Создаем DataFrame из списка словарей
        # Pandas автоматически использует ключи словарей как названия колонок
        df = pd.DataFrame(report_data)

        # 2. (Опционально) Задаем желаемый порядок колонок, если он важен
        # Если какой-то колонки не будет в данных, она будет проигнорирована
        desired_columns = [
            "Дата",
            "Подразделение",
            "Операция",
            "Культура",
            "За день, га",
            "С начала операции, га",
            "Вал за день, ц",
            "Вал с начала, ц"
        ]
        # Оставляем только те колонки из desired_columns, которые реально есть в DataFrame
        # и сохраняем исходный порядок остальных колонок, если они есть
        existing_columns = [col for col in desired_columns if col in df.columns]
        other_columns = [col for col in df.columns if col not in existing_columns]
        df = df[existing_columns + other_columns] # Переупорядочиваем колонки

        # 3. Сохраняем DataFrame в Excel файл
        # index=False предотвращает запись индекса DataFrame (0, 1, 2...) как отдельной колонки в Excel
        df.to_excel(filename, index=False)
        print(f"Отчет успешно сохранен в файл: {filename}")

    except ImportError:
         print("Ошибка: Библиотека pandas не установлена. Пожалуйста, установите ее (pip install pandas).")
    except Exception as e:
        print(f"Произошла ошибка при сохранении в Excel: {e}")

In [62]:
# --- Финальная JSON схема в виде словаря Python ---
final_schema_dict = {
  "type": "object",
  "properties": {
    "reports": {
      "type": "array",
      "description": "Список извлеченных отчетов по работам.",
      "items": {
        "type": "object",
        "properties": {
          "Дата": {
            "type": ["string", "null"], # Используем массив типов
            "description": "Дата отчета в формате YYYY-MM-DD."
          },
          "Подразделение": {
            "type": ["string", "null"],
            "description": "Название подразделения (без ПУ/Отделения)."
          },
          "Операция": {
            "type": ["string", "null"],
            "description": "Полное название полевой работы из справочника."
          },
          "Культура": {
            "type": ["string", "null"],
            "description": "Полное название культуры из справочника."
          },
          "За день, га": {
            "type": ["number", "null"], # Тип number включает integer
            "description": "Количество гектар за день."
          },
          "С начала операции, га": {
            "type": ["number", "null"],
            "description": "Общее количество гектар с начала операции."
          },
          "Вал за день, ц": {
            "type": ["number", "null"],
            "description": "Валовый сбор за день в центнерах."
          },
          "Вал с начала, ц": {
            "type": ["number", "null"],
            "description": "Валовый сбор с начала операции в центнерах."
          }
        },
        "required": [
          "Дата",
          "Подразделение",
          "Операция",
          "Культура",
          "За день, га",
          "С начала операции, га",
          "Вал за день, ц",
          "Вал с начала, ц"
        ],
        "additionalProperties": False
      }
    }
  },
  "required": [
    "reports"
  ],
  "additionalProperties": False
}


# --- Упрощенный промпт (остается без изменений) ---
SIMPLIFIED_AGRO_REPORT_PROMPT_TEMPLATE = """
Проанализируй следующее сообщение с отчетом о сельскохозяйственных работах:
---
{input_message}
---

Используй следующие справочники для распознавания терминов и определения значений:

СПИСОК КУЛЬТУР:
{cultures_list}

СПИСОК ОПЕРАЦИЙ:
{operations_list}

СПИСОК ПОДРАЗДЕЛЕНИЙ:
{departments_list}

ЗАДАЧА:
1.  Найди **все** уникальные комбинации (Операция, Культура, Подразделение), логически связанные в тексте сообщения.
2.  Для **каждой** такой комбинации извлеки связанные с ней числовые данные: "За день, га", "С начала операции, га", "Вал за день, ц", "Вал с начала, ц".
3.  Определи **дату** отчета. Сначала попробуй найти дату в тексте сообщения (например, "25.07", "25 июля"). Если дата в тексте не найдена, используй текущую дату: {current_date}.

ИНСТРУКЦИИ ПО ИЗВЛЕЧЕНИЮ ДЛЯ КАЖДОГО ОБЪЕКТА В СПИСКЕ:
- "Дата": Дата отчета в формате YYYY-MM-DD (извлеченная из текста или текущая {current_date}).
- "Подразделение": Определи название подразделения для данной комбинации. Используй номер отделения (Отд), чтобы найти соответствующее подразделение в "СПИСКЕ ПОДРАЗДЕЛЕНИЙ". Если есть несколько "Отд", относящихся к одной записи, ориентируйся на первое или на общие данные по ПУ (Производственному участку), если они есть. Если номер отделения не указан или не найден, попробуй найти название подразделения явно в тексте рядом с операцией/культурой. Если определить не удается, используй null. Не включай ПУ или Отделение в название.
- "Операция": Определи полное название полевой работы из "СПИСКА ОПЕРАЦИЙ", основываясь на тексте, связанном с этой комбинацией (например, "Предп культ" -> "Предпосевная культивация"). Если не указана или не распознана для этой комбинации, используй null.
- "Культура": Определи полное название культуры из "СПИСКА КУЛЬТУР", основываясь на тексте, связанном с этой комбинацией (например, "оз пш" -> "Пшеница озимая"). Если не указана или не распознана для этой комбинации, используй null.
- "За день, га": Количество гектар за день (число перед '/'). Если есть данные по "Отд", суммируй их. Если есть данные "По Пу" и "Отд", используй данные "По Пу". Если не указано, используй null.
- "С начала операции, га": Общее количество гектар с начала операции (число после '/'). Если есть данные по "Отд", суммируй их. Если есть данные "По Пу" и "Отд", используй данные "По Пу". Если не указано, используй null.
- "Вал за день, ц": Валовый сбор за день в центнерах. Если не указано, используй null.
- "Вал с начала, ц": Валовый сбор с начала операции в центнерах. Если не указано, используй null.

ОБРАТИ ВНИМАНИЕ: 
1. Если в сообщении указано:
Пахота зяби под мн тр
По Пу 26/488
Отд 12 26/221
то бери информацию только по ПУ - так как это данные сразу по производственному участку, а все строчки где указано "Отд" относятся к отдельным отделениям. (которые суммируются в случае, если значение "По Пу" не указано)
2. Если в сообщении указано:
Диск к. Сил отд 7. 32/352
Пу- 484
Это значит, что отделение сделало 32 гектара за день, 352 с начала операции, а по производственному участку (ПУ) - 484 гектара. Значит надо выносить цифры 32 гектра за день и 484 гектара с начала операции.


ВАЖНО:
- Если сообщение не содержит данных о работах или их не удается извлечь, верни пустой JSON объект вида {{"reports": []}}.
- Тебе нужно вернуть JSON объект, соответствующий предоставленной JSON schema.
"""


In [64]:
# --- Обновленная функция с использованием client.responses.create ---
def openai_extract_reports_manual_schema(client: OpenAI,
                                           input_message: str,
                                           cultures_list: str,
                                           operations_list: str,
                                           departments_list: str,
                                           model_name: str
                                           ) -> list[dict] | None: # Возвращаем list[dict]
    """
    Извлекает структурированные данные из агро-отчета с помощью OpenAI Responses API
    и вручную созданной JSON схемы.

    Args:
        client: Инициализированный клиент OpenAI.
        input_message: Строка с текстом сообщения.
        cultures_list: Строка со списком культур.
        operations_list: Строка со списком операций.
        departments_list: Строка со списком подразделений (JSON).
        model_name: Название модели OpenAI.

    Returns:
        Список словарей с данными отчетов или None в случае ошибки.
    """
    current_date = datetime.date.today().isoformat()

    # Формируем промпт
    try:
        prompt = SIMPLIFIED_AGRO_REPORT_PROMPT_TEMPLATE.format(
            input_message=input_message,
            cultures_list=cultures_list,
            operations_list=operations_list,
            departments_list=departments_list,
            current_date=current_date
        )
    except KeyError as e:
        print(f"Ошибка форматирования промпта: не найден ключ {e}")
        print("Убедитесь, что в шаблоне SIMPLIFIED_AGRO_REPORT_PROMPT_TEMPLATE нет лишних или неправильных фигурных скобок.")
        return None

    # Вызываем API
    try:
        response = client.responses.create(
            model=model_name,
            input=[
                {"role": "system", "content": "You are an AI assistant designed to extract structured data from agricultural reports according to a specific JSON schema."},
                {"role": "user", "content": prompt} # Передаем отформатированный промпт
            ],
            text={
                "format": {
                    "type": "json_schema",
                    "name": "report_schema", # Имя схемы
                    "strict": True,
                    "schema": final_schema_dict # Используем финальный словарь схемы
                }
            }
            # Убираем необязательные параметры для чистоты, можно добавить при необходимости
            # temperature=1,
            # max_output_tokens=3933,
            # top_p=0.6,
        )

        # Проверяем статус ответа
        if response.status == "incomplete":
            reason = response.incomplete_details.reason if response.incomplete_details else "unknown"
            print(f"Ошибка: Ответ не завершен по причине: {reason}")
            return None
        elif response.status != "completed":
            error_details = response.error if response.error else "Нет деталей"
            print(f"Ошибка: Неожиданный статус ответа: {response.status}. Детали: {error_details}")
            return None

        # Проверяем отказ
        # Обратите внимание: структура output может немного отличаться, добавим проверки
        if not response.output or not response.output[0].content:
             print("Ошибка: Неожиданная структура ответа, отсутствует output или content.")
             print("Полный ответ:", response)
             return None

        output_content = response.output[0].content[0]

        if output_content.type == "refusal":
            refusal_message = output_content.refusal
            print(f"Ошибка: Модель отказалась выполнять запрос: {refusal_message}")
            return None

        # Извлекаем текст
        if output_content.type != "output_text":
             print(f"Ошибка: Ответ не содержит ожидаемый тип 'output_text', получен тип '{output_content.type}'.")
             print("Полный ответ:", response)
             return None

        raw_response_content = output_content.text

        # Парсим JSON
        try:
            parsed_response_obj = json.loads(raw_response_content)
            if isinstance(parsed_response_obj, dict) and "reports" in parsed_response_obj:
                 parsed_data = parsed_response_obj.get('reports') # Используем .get для безопасности
                 if not isinstance(parsed_data, list):
                      print("Ошибка: Ключ 'reports' в ответе не содержит список.")
                      print("Ответ модели был:", raw_response_content)
                      return None
            else:
                 print("Ошибка: Ответ модели не содержит ожидаемый ключ 'reports' или не является словарем.")
                 print("Ответ модели был:", raw_response_content)
                 return None

        except json.JSONDecodeError as e:
            print(f"\nОшибка парсинга JSON: {e}")
            print("Ответ модели был:")
            print(raw_response_content)
            return None

        # Добавляем дату, если она null
        for item in parsed_data:
            if isinstance(item, dict) and item.get("Дата") is None:
                item["Дата"] = current_date

        return parsed_data

    except Exception as e:
        # Ловим все остальные ошибки (включая ошибки API, например, BadRequestError)
        print(f"\nПроизошла общая ошибка при вызове OpenAI Responses API или обработке ответа: {e}")
        print(traceback.format_exc()) # Выводим полный traceback
        return None


In [68]:
test_message = """
"Пахота под сах св
По Пу 88/329
Отд 11 23/60
Отд 12 34/204
Отд 16 31/65

Пахота под мн тр
По Пу 10/438
Отд 17 10/80

Чизел под оз ячмень
По Пу 71/528
Отд 11 71/130

2-е диск под сах св
По Пу 80/1263
Отд 12 80/314

2-е диск под оз ячмень
По Пу 97/819
Отд 17 97/179

Диск кук силос
По Пу 43/650
Отд 11 33/133
Отд 12 10/148

Выкаш отц форм под/г
Отд 12 10/22

Уборка сах св
Отд 12 16/16
Вал 473920
Урож 296,2
Диг - 19,19
Оз - 5,33"
"""
model="gpt-4.1-mini"

analysis_result = openai_extract_reports_manual_schema(
    client=openai_client,                      # <-- Передаем инициализированный клиент OpenAI
    input_message=test_message,
    cultures_list=cultures_list,
    operations_list=operations_list,
    departments_list=departments_list,
    model_name=model         
)


# analysis_result = deepseek_extract_reports(
#     input_message=test_message,
#     client=deepseek_client,
#     cultures_list=cultures_list,       # Справочник культур
#     operations_list=operations_list,   # Справочник операций
#     departments_list=departments_list, # Справочник подразделений (используем departments_data как в исходном коде)
#     prompt_template=DETAILED_AGRO_REPORT_PROMPT_TEMPLATE # Передаем шаблон промпта
# )

# Вывод результата
analysis_result

[{'Дата': '2025-04-15',
  'Подразделение': 'АОР Юг',
  'Операция': 'Пахота',
  'Культура': 'Свекла сахарная',
  'За день, га': 88,
  'С начала операции, га': 329,
  'Вал за день, ц': None,
  'Вал с начала, ц': None},
 {'Дата': '2025-04-15',
  'Подразделение': 'АОР Юг',
  'Операция': 'Пахота',
  'Культура': 'Многолетние травы текущего года',
  'За день, га': 10,
  'С начала операции, га': 438,
  'Вал за день, ц': None,
  'Вал с начала, ц': None},
 {'Дата': '2025-04-15',
  'Подразделение': 'АОР Юг',
  'Операция': 'Чизлевание',
  'Культура': 'Ячмень озимый',
  'За день, га': 71,
  'С начала операции, га': 528,
  'Вал за день, ц': None,
  'Вал с начала, ц': None},
 {'Дата': '2025-04-15',
  'Подразделение': 'АОР Юг',
  'Операция': 'Дискование 2-е',
  'Культура': 'Свекла сахарная',
  'За день, га': 80,
  'С начала операции, га': 1263,
  'Вал за день, ц': None,
  'Вал с начала, ц': None},
 {'Дата': '2025-04-15',
  'Подразделение': 'АОР Юг',
  'Операция': 'Дискование 2-е',
  'Культура': 'Ячмен

In [ ]:
# Промпт генератор

# test_message = """
# "Пахота под сах св
# По Пу 88/329
# Отд 11 23/60
# Отд 12 34/204
# Отд 16 31/65

# Пахота под мн тр
# По Пу 10/438
# Отд 17 10/80

# Чизел под оз ячмень
# По Пу 71/528
# Отд 11 71/130

# 2-е диск под сах св
# По Пу 80/1263
# Отд 12 80/314

# 2-е диск под оз ячмень
# По Пу 97/819
# Отд 17 97/179

# Диск кук силос
# По Пу 43/650
# Отд 11 33/133
# Отд 12 10/148

# Выкаш отц форм под/г
# Отд 12 10/22

# Уборка сах св
# Отд 12 16/16
# Вал 473920
# Урож 296,2
# Диг - 19,19
# Оз - 5,33"
# """

# current_date = datetime.date.today().isoformat()

# prompt = SIMPLIFIED_AGRO_REPORT_PROMPT_TEMPLATE.format(
#     input_message=test_message,
#     cultures_list=cultures_list,
#     operations_list=operations_list,
#     departments_list=departments_list,
#     current_date=current_date
# )   
# prompt

In [55]:
# analysis_result = gemini_extract_reports(
#     input_message=test_message,
#     cultures_list=cultures_list,       # Справочник культур
#     operations_list=operations_list,   # Справочник операций
#     departments_list=departments_list, # Справочник подразделений (используем departments_data как в исходном коде)
#     model=model,                       # Модель Gemini
#     prompt_template=DETAILED_AGRO_REPORT_PROMPT_TEMPLATE # Передаем шаблон промпта
# )
# # Вывод результата
# analysis_result

In [ ]:
# Генерируем имя файла с текущей датой для уникальности
today_str = datetime.date.today().strftime('%Y-%m-%d')
output_filename = f'agro_report_{today_str}.xlsx'

# Вызываем нашу новую функцию для сохранения данных
save_reports_to_excel(analysis_result, output_filename)

Известные проблемы: 

надо как-то центнеры округлять до кг в сообщении из строчки 50
Не всего корректно считывается вал 